In [127]:
import mne
from mne.viz import circular_layout
from mne_connectivity import spectral_connectivity_epochs
from mne_connectivity.viz import plot_connectivity_circle
from mne_connectivity.viz import plot_sensors_connectivity
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# read one sample data
mne_filename = "mne_data/C10_32Ch_48Subjects_al_raw_eeg.fif"
raw = mne.io.read_raw_fif(mne_filename, verbose=False)

In [3]:
events = mne.make_fixed_length_events(raw, start=0, stop=20, duration=2., overlap=1.75, id = 1)


In [6]:
epochs = mne.make_fixed_length_epochs(raw=raw, duration=2, overlap=0.75)

Not setting metadata
15 matching events found
No baseline correction applied
0 projection items activated


In [151]:
eeg_montage = mne.channels.make_standard_montage('standard_1020')

# map the sensor locations from montage
for i, ch_name in enumerate(epochs.info['ch_names']):
    # print(i, ch_name)
    for j, ch_name_montage in enumerate(eeg_montage.ch_names):
        if ch_name.lower() == ch_name_montage.lower():
            # print(j, ch_name_montage)
            epochs.info['chs'][i]['loc'] = np.append(eeg_montage.dig[j]['r'], [0]*9)

In [150]:
eeg_montage.ch_names[0]

'Fp1'

In [153]:
# Compute connectivity for band containing the evoked response.
# We exclude the baseline period:
fmin, fmax = 4., 9.
sfreq = raw.info['sfreq']  # the sampling frequency
tmin = 0.0  # exclude the baseline period
epochs.load_data() 
con = spectral_connectivity_epochs(
    epochs, method='pli', mode='multitaper', sfreq=sfreq, fmin=fmin, fmax=fmax,
    faverage=True, tmin=tmin, mt_adaptive=False, n_jobs=1)

# fmin = (0.5, 4., 8., 13., 30.)
# fmax = (4., 8., 13., 30., 45.)

fmin = 4.
fmax = 9.
sfreq = raw.info['sfreq']  # the sampling frequency
con_methods = ['pli', 'wpli2_debiased', 'ciplv']
con = spectral_connectivity_epochs(
    epochs, method="wpli2_debiased", mode='multitaper', sfreq=sfreq, fmin=fmin,
    fmax=fmax, faverage=True, mt_adaptive=True, n_jobs=1)

# Now, visualize the connectivity in 3D:
plot_sensors_connectivity(
    epochs.info,
    con.get_data(output='dense')[:,:,0], picks=None)

Replacing existing metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..1.996s for estimation (512 points)
    frequencies: 4.0Hz..9.0Hz (11 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connectivity for epoch 11
    computing connectivity for epoch 12
    computing connectivity for epoch 13
    computing connectivity for epoch 14
    computing connectiv

In [49]:
builtin_montages = mne.channels.get_builtin_montages()
for montage in builtin_montages:
    print(montage)

EGI_256
GSN-HydroCel-128
GSN-HydroCel-129
GSN-HydroCel-256
GSN-HydroCel-257
GSN-HydroCel-32
GSN-HydroCel-64_1.0
GSN-HydroCel-65_1.0
biosemi128
biosemi16
biosemi160
biosemi256
biosemi32
biosemi64
easycap-M1
easycap-M10
mgh60
mgh70
standard_1005
standard_1020
standard_alphabetic
standard_postfixed
standard_prefixed
standard_primed
artinis-octamon
artinis-brite23


In [52]:
eeg_montage = mne.channels.make_standard_montage('standard_1020')